# Chapter 16: Multiloop and Multivariable Control

## Example 16.1: Pilot_scale_distillation_column.sce

In [ ]:
clear
clc
//Example 16.1
disp('Example 16.1')
K=[12.8 -18.9;6.6 -19.4];
tau=[16.7 21;10.9 14.4];
s=%s;
G=K./(1+tau*s);
//ITAE settings from Table 11.3
K1=12.8;tau1=16.7;theta1=1;K2=-19.4;tau2=14.4;theta2=3;
Kc1=1/K1*0.586*(theta1/tau1)^-0.916;taui1=tau1*inv(1.03-0.165*(theta1/tau1)); 
Kc2=1/K2*0.586*(theta2/tau2)^-0.916;taui2=tau2*inv(1.03-0.165*(theta2/tau2)); 
mprintf('          Kc          tauI')
mprintf('
x_D-R   %f    %f',Kc1,taui1)
mprintf('
x_B-R   %f    %f',Kc2,taui2)
Kc=[Kc1;Kc2];
tauI=[taui1;taui2];
//====Making step response models of the continuos transfer functions====//
Ts=0.1;//Sampling time ie delta_T
delay3=3/Ts;
delay1=1/Ts;
delay7=7/Ts;
N=100/Ts;//Model Order
s=%s;
G=syslin('c',diag(matrix(G,1,4)));//Transfer function
t=0:Ts:N*Ts;
u_sim=ones(4,length(t));
//Modeling Output delays through input delay in steps
u_sim(1,1:(delay1))=zeros(1,delay1); 
u_sim(3,1:(delay7))=zeros(1,delay7); 
u_sim([2 4],1:(delay3))=zeros(2,delay3);
S=csim(u_sim,t,G)';//generating step response model for real plant
//plot(t,S);
S(1,:)=[];
//Now we have these step response models for each of the transfer functions
//[S1 S3
//S2 S4 
T=120;//Simulation Run Time in minutes
n=T/Ts*2+1; //no. of discrete points in our domain of analysis
//===============================================================//
//===============================================================//
//===============================================================//
//========Set point as +1 in X-D==X-B loop in manual=============//
//p is the controller output
p=zeros(n,2);
delta_p=zeros(n,2);
e=zeros(n,2); //errors=(ysp-y) on which PI acts
ysp=zeros(n,2);
ysp((n-1)/2+1:n,1)=ones(n-((n-1)/2+1)+1,1);
t=-(n-1)/2*Ts:Ts:(n-1)/2*Ts;
y=zeros(n,2);
for k=(n-1)/2+1:n
    
    //Error  e
    e(k,:)=ysp(k-1,:)-y(k-1,:);
    delta_e(k,:)=e(k,:)-e(k-1,:);
    //Controller calculation----Digital PID----Eqn 7-28 Pg 136 (Velocity form)
    p(k,1)=p(k-1,1)+([delta_e(k,1)+e(k,1)*diag(Ts/taui1)]*diag(Kc1));
    //1-1/2-2 pairing
    
    delta_p(k,:)=p(k,:)-p(k-1,:);
       
    //Output
    y(k,1)=[S(1:N-1,1);S(1:N-1,3)]'...
        *[flipdim(delta_p(k-N+1:k-1,1),1);flipdim(delta_p(k-N+1:k-1,2),1)]...
        +[S(N,1) S(N,3)]*[p(k-N,1);p(k-N,2)];
    y(k,2)=[S(1:N-1,2);S(1:N-1,4)]'...
        *[flipdim(delta_p(k-N+1:k-1,1),1);flipdim(delta_p(k-N+1:k-1,2),1);]...
        +[S(N,2) S(N,4)]*[p(k-N,1);p(k-N,2)];
end
subplot(2,1,1);
plot(t',y(:,1),'b-');
set(gca(),'data_bounds',[0 40 0 1.4]); //putting bounds on display
l=legend('$x_B	ext{ loop in manual}$',position=1);
xtitle('','Time(min)','$x_D$');
a=get('current_axes');
c=a.y_label;c.font_size=5;
//===============================================================//
//===============================================================//
//===============================================================//
//========Set point as +1 in X-B==X-D loop in manual=============//
//p is the controller output
p=zeros(n,2);
delta_p=zeros(n,2);
e=zeros(n,2); //errors=(ysp-y) on which PI acts
ysp=zeros(n,2);
ysp((n-1)/2+1:n,2)=ones(n-((n-1)/2+1)+1,1);
t=-(n-1)/2*Ts:Ts:(n-1)/2*Ts;
y=zeros(n,2);
for k=(n-1)/2+1:n
    
    //Error  e
    e(k,:)=ysp(k-1,:)-y(k-1,:);
    delta_e(k,:)=e(k,:)-e(k-1,:);
    //Controller calculation----Digital PID----Eqn 7-28 Pg 136 (Velocity form)
    p(k,2)=p(k-1,2)+([delta_e(k,2)+e(k,2)*diag(Ts/taui2)]*diag(Kc2));
    //1-1/2-2 pairing
    
    delta_p(k,:)=p(k,:)-p(k-1,:);
       
    //Output
    y(k,1)=[S(1:N-1,1);S(1:N-1,3)]'...
        *[flipdim(delta_p(k-N+1:k-1,1),1);flipdim(delta_p(k-N+1:k-1,2),1)]...
        +[S(N,1) S(N,3)]*[p(k-N,1);p(k-N,2)];
    y(k,2)=[S(1:N-1,2);S(1:N-1,4)]'...
        *[flipdim(delta_p(k-N+1:k-1,1),1);flipdim(delta_p(k-N+1:k-1,2),1);]...
        +[S(N,2) S(N,4)]*[p(k-N,1);p(k-N,2)];
end
subplot(2,1,2);
plot(t',y(:,2),'b-');
set(gca(),'data_bounds',[0 40 0 1.4]); //putting bounds on display
l=legend('$x_D	ext{ loop in manual}$',position=1);
xtitle('','Time(min)','$x_B$');
a=get('current_axes');
c=a.y_label;c.font_size=5;
//===============================================================//
//===============================================================//
//===============================================================//
//========Set point as +1 in X-D==Both loops Automatic===========//
//p is the controller output
p=zeros(n,2);
delta_p=zeros(n,2);
e=zeros(n,2); //errors=(ysp-y) on which PI acts
ysp=zeros(n,2);
ysp((n-1)/2+1:n,1)=ones(n-((n-1)/2+1)+1,1);
t=-(n-1)/2*Ts:Ts:(n-1)/2*Ts;
y=zeros(n,2);
for k=(n-1)/2+1:n
    
    //Error  e
    e(k,:)=ysp(k-1,:)-y(k-1,:);
    delta_e(k,:)=e(k,:)-e(k-1,:);
    //Controller calculation----Digital PID----Eqn 7-28 Pg 136 (Velocity form)
//    p(k,:)=p(k-1,:)+flipdim([delta_e(k,:)+e(k,:)*diag(Ts./tauI)]*diag(Kc),2);
    p(k,:)=p(k-1,:)+([delta_e(k,:)+e(k,:)*diag(Ts./tauI)]*diag(Kc));
    //1-1/2-2 pairing
    
    delta_p(k,:)=p(k,:)-p(k-1,:);
       
    //Output
    y(k,1)=[S(1:N-1,1);S(1:N-1,3)]'...
        *[flipdim(delta_p(k-N+1:k-1,1),1);flipdim(delta_p(k-N+1:k-1,2),1)]...
        +[S(N,1) S(N,3)]*[p(k-N,1);p(k-N,2)];
    y(k,2)=[S(1:N-1,2);S(1:N-1,4)]'...
        *[flipdim(delta_p(k-N+1:k-1,1),1);flipdim(delta_p(k-N+1:k-1,2),1);]...
        +[S(N,2) S(N,4)]*[p(k-N,1);p(k-N,2)];
end
subplot(2,1,1);
plot(t',y(:,1),'r--');
set(gca(),'data_bounds',[0 40 0 1.4]); //putting bounds on display
l=legend('$x_B	ext{ loop in manual}$','$	ext{Both loops in automatic}$',position=1);
//l.font_size=5;
xtitle('','Time(min)','$x_D$');
a=get('current_axes');
c=a.y_label;c.font_size=5;
//===============================================================//
//===============================================================//
//===============================================================//
//========Set point as +1 in X-B==Both loops Automatic===========//
//p is the controller output
p=zeros(n,2);
delta_p=zeros(n,2);
e=zeros(n,2); //errors=(ysp-y) on which PI acts
ysp=zeros(n,2);
ysp((n-1)/2+1:n,2)=ones(n-((n-1)/2+1)+1,1);
t=-(n-1)/2*Ts:Ts:(n-1)/2*Ts;
y=zeros(n,2);
for k=(n-1)/2+1:n
    
    //Error  e
    e(k,:)=ysp(k-1,:)-y(k-1,:);
    delta_e(k,:)=e(k,:)-e(k-1,:);
    //Controller calculation----Digital PID----Eqn 7-28 Pg 136 (Velocity form)
//    p(k,:)=p(k-1,:)+flipdim([delta_e(k,:)+e(k,:)*diag(Ts./tauI)]*diag(Kc),2);
    p(k,:)=p(k-1,:)+([delta_e(k,:)+e(k,:)*diag(Ts./tauI)]*diag(Kc));
    //1-1/2-2 pairing
    
    delta_p(k,:)=p(k,:)-p(k-1,:);
       
    //Output
    y(k,1)=[S(1:N-1,1);S(1:N-1,3)]'...
        *[flipdim(delta_p(k-N+1:k-1,1),1);flipdim(delta_p(k-N+1:k-1,2),1)]...
        +[S(N,1) S(N,3)]*[p(k-N,1);p(k-N,2)];
    y(k,2)=[S(1:N-1,2);S(1:N-1,4)]'...
        *[flipdim(delta_p(k-N+1:k-1,1),1);flipdim(delta_p(k-N+1:k-1,2),1);]...
        +[S(N,2) S(N,4)]*[p(k-N,1);p(k-N,2)];
end
subplot(2,1,2);
plot(t',y(:,2),'r--');
set(gca(),'data_bounds',[0 40 0 1.4]); //putting bounds on display
l=legend('$x_D	ext{ loop in manual}$','$	ext{Both loops in automatic}$',position=1);
xtitle('','Time(min)','$x_B$');
a=get('current_axes');
c=a.y_label;c.font_size=5;
//Also refer to Example 22.4 for similar application of algorithm of multiploop PID

## Example 16.6: Sensitivity_of_steady_state_gain_matrix.sce

In [ ]:
clear
clc
//Example 16.6
disp('Example 16.6')
K1=[1 0;10 1];//K with K12=0
eig1=spec(K1);
sigma1=spec(K1'*K1);
CN1=sqrt(max(sigma1)/min(sigma1))
mprintf('
Eigenvalues of K1 are %f and %f
 and CN is %f',eig1',CN1)
K2=[1 0.1;10 1];//K with K12=0.1
eig2=spec(K2);
sigma2=spec(K2'*K2);
CN2=sqrt(max(sigma2)/min(sigma2))
mprintf('
Eigenvalues of K2 are %f and %f
 and CN is %f',eig2',CN2)

## Example 16.7: Preferred_multiloop_control_strategy.sce

In [ ]:
clear
clc
//Example 16.7
disp('Example 16.7')
X=[0.48 0.9 -0.006;0.52 0.95 0.008; 0.90 -0.95 0.020];
[U,S,V]=svd(X)
RGA=X.*([inv(X)]')  //Eqn 16-36
//Condition no. of X
CN=max(diag(S))/min(diag(S))
//Note that condition no. can also be found with command cond(X)
// The RGA given in the book is wrong! Eqn 16-73 is wrong.
mprintf('
 The RGA given in the book is wrong! Eqn 16-73 is wrong.
')
disp(RGA,'RGA=')
X1=X(1:2,1:2);
X2=X(1:2,[1 3]);
X3=X(1:2,2:3);
X4=X([1 3],1:2);
X5=X([1 3],[1 3]);
X6=X([1 3],2:3);
X7=X([2 3],1:2);
X8=X([2 3],[1 3]);
X9=X([2 3],2:3);
lamda1=max(X1.*inv(X1'));
lamda2=max(X2.*inv(X2'));
lamda3=max(X3.*inv(X3'));
lamda4=max(X4.*inv(X4'));
lamda5=max(X5.*inv(X5'));
lamda6=max(X6.*inv(X6'));
lamda7=max(X7.*inv(X7'));
lamda8=max(X8.*inv(X8'));
lamda9=max(X9.*inv(X9'));
mprintf('
 Pairing no.             CN        lambda
')
mprintf('
 1             %f        %f',cond(X1),lamda1)
mprintf('
 2             %f        %f',cond(X2),lamda2)
mprintf('
 3             %f        %f',cond(X3),lamda3)
mprintf('
 4             %f        %f',cond(X4),lamda4)
mprintf('
 5             %f        %f',cond(X5),lamda5)
mprintf('
 6             %f        %f',cond(X6),lamda6)
mprintf('
 7             %f        %f',cond(X7),lamda7)
mprintf('
 8             %f        %f',cond(X8),lamda8)
mprintf('
 9             %f        %f',cond(X9),lamda9)